In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

IMG_SIZE = 224
BATCH_SIZE = 32
INITIAL_EPOCHS = 20  # Initial epochs for the pre-trained model
FINAL_EPOCHS = 40    # Total epochs including fine-tuning
LEARNING_RATE = 0.0001

# Data directories
train_data_dir = 'D:\\Broken Dataset\\Training'
test_data_dir = 'D:\\Broken Dataset\\Validation'

# Data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

# Load pre-trained model
pretrained_model = load_model('path_to_pretrained_model.h5')

# Freeze layers of the pre-trained model
for layer in pretrained_model.layers:
    layer.trainable = False

x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
predictions = Dense(2, activation='softmax')(x)

# Creating model
model = Model(inputs=pretrained_model.input, outputs=predictions)

optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model with additional epochs
history = model.fit(train_generator, epochs=FINAL_EPOCHS, initial_epoch=INITIAL_EPOCHS, validation_data=test_generator)
